In [ ]:
import json
import csv
import sys  


with open('/datadrive/urbanplayground/Streetview/Traffic/CD/cdtraffic.csv', 'w', newline='') as f:
	writer = csv.writer(f)
	with open('/datadrive/urbanplayground/Streetview/Traffic/CD/201904175.csv', 'r') as sdata:
		sreader = csv.reader(sdata)
		for srow in sreader:
			i=sreader.line_num-1
			print(i)
			for filenum in range(5,21):
				filepath='/datadrive/urbanplayground/Streetview/Traffic/CD/20190417'+str(filenum)+'.csv' 
				with open(filepath, 'r') as data:
					lines = data.readlines()
					# Get coordinate
					#cspeed=lines[i].rstrip('\r').split(',')[5]
					ffspeed=lines[i].rstrip('\r').split(',')[5]
					#i=i+2
					ffspeed=ffspeed.strip('\t\r\n')
					#srow.append(cspeed)
					srow.append(ffspeed)
			for filenum in range(5,21):
				filepath='/datadrive/urbanplayground/Streetview/Traffic/CD/20190420'+str(filenum)+'.csv' 
				with open(filepath, 'r') as data:
					lines = data.readlines()
					# Get coordinate
					#cspeed=lines[i].rstrip('\r').split(',')[5]
					ffspeed=lines[i].rstrip('\r').split(',')[5]
					#i=i+2
					ffspeed=ffspeed.strip('\t\r\n')
					#srow.append(cspeed)
					srow.append(ffspeed)
			writer.writerow(srow)

In [ ]:
import numpy as np
import csv
import time

with open("./bjtraffic.csv",'rt',encoding='UTF-8') as raw_data:
    readers=csv.reader(raw_data,delimiter=',')
    x=list(readers)
    traffic=np.array(x)
    pt = np.loadtxt("./BJPtDB.csv",dtype=np.str,delimiter=',',encoding='utf-8')
    with open('./BJ.csv', 'w') as fileWriteObj:
        roadidt = pt[0][6]
        count = 0
        lcount = 0
        for j in range(0,pt.size):
            #print(pt[j])
            time.sleep(1)
            row = []
            roadid = pt[j][6]
            num = pt[j][0]
            road_type = pt[j][8]
            lng = pt[j][4]
            lat = pt[j][5]
            dem = pt[j][8]
            slope = pt[j][9]
            aspect = pt[j][10]
            #print("j num")
            #print(j,num)
            row = [num,roadidt,road_type,lng,lat,dem,slope,aspect]
            if roadid != roadidt:
                #print("unequal")
                #print(num,traffic[lcount][0])
                count = 1
                if num != traffic[lcount][0]:
                    lcount += 1
                roadidt = roadid 
            else:
                # print("equal")
                count += 1
            if count%3 == 0:
                #print(lcount,traffic[lcount][0])
                for i in range(5,len(traffic[lcount])):
                    if(traffic[lcount][i]==-1):
                        row.append(1)
                    else:
                        row.append((100.0-float(traffic[lcount][i]))/100.0)
                lcount += 1
            else:
                #print(lcount,traffic[lcount][0])
                for i in range(5,len(traffic[lcount])):
                    if(traffic[lcount][i]==-1):
                        row.append(1)
                    else:
                        row.append((100.0-float(traffic[lcount][i]))/100.0)
            writer = csv.writer(fileWriteObj, lineterminator='\n')
            writer.writerow(row)

In [3]:
import requests, json
from queue import Queue
from threading import Thread
import numpy as np
import pandas as pd
import time, random
import traceback
import csv
import math

class Spider():
    def __init__(self):
        self.thread_num = 4
        self.city = 'BJ'
        self.listall = []
        self.start = time.time()
        self.groups_q = self.get_points()
        self.lenth = self.groups_q.qsize()
        self.aks_q = self.get_aks()
        self.valid_q = Queue()

    def get_aks(self):
        with open("/datadrive/lzy/Traffic/BaiduAK.txt", 'r', encoding='utf8') as f:
            apilines = f.readlines()
        aks_q = Queue()
        for ak in apilines:
            aks_q.put(ak)
        return aks_q
    
    def split_list(self, l, n=100):
        # 均分列表
        new_l = []
        for i in range(0,len(l),n):
            new_l.append(l[i:i+n])
        new_l.append(l[int(len(l)/n)*n:])
        return new_l
    
    def get_points(self):
        df_p = pd.read_csv('/datadrive/lzy/Traffic/'+ self.city +'/'+ self.city +'AllPoints.csv')
        df_p.rename(columns={'p_id':'fid'},inplace=True)
        df_p.info()
        df_R = pd.read_csv('/datadrive/lzy/R/'+ self.city +'BSV_r.csv')
        df_R.info()
        df=pd.merge(df_p,df_R,on = 'fid',how='right')
        groups_q = Queue()
        groups = np.array(df.iloc[:,[0,3,4]]).tolist()
        point_groups = self.split_list(groups)
        for group in point_groups:
            groups_q.put(group)
        return groups_q
    
    def TileCoord(self,point,zoom):
        pointx,pointy = point.split(",")
        Xtemp = float(pointx)* math.pow(2,zoom - 18)
        Ytemp = float(pointy)* math.pow(2,zoom - 18)
        # print(Xtemp,Ytemp)
        tileX = math.floor(Xtemp / 256)
        tileY = math.floor(Ytemp / 256)
        pixelX = Xtemp - tileX * 256
        pixelY = Ytemp - tileY * 256
        return [tileX, tileY, pixelX, pixelY]


    def req(self, group):
        coordids = []
        w84c = []
        for point in group:
            coordids.append(point[0])
            lon = point[1]
            lat = point[2]                   
            # 之前返回的结果里的 XY 坐标是*100的，所以这里要缩放一下
            w84c.append("{},{}".format(lon, lat))
        # 访问 url 直到得到正确的访问结果
        while True:
            time.sleep(random.random()*2)
            ak = self.aks_q.get()
            ak = ak.strip('\t\r\n')
            data = {
                "ak": ak,
                "coords": "{}".format(";".join(w84c)),
                "from": "1",
                "to": "6",
                }
            data2 = {
                "ak": ak,
                "coords": "{}".format(";".join(w84c)),
                "from": "1",
                "to": "5",
                }
            try:
                # request coordinations
                r = requests.get("http://api.map.baidu.com/geoconv/v1/?", params=data,timeout=60)
                r.encoding='utf-8'
                data = r.json()
                print(ak)
                r2 = requests.get("http://api.map.baidu.com/geoconv/v1/?", params=data2,timeout=60)
                r2.encoding='utf-8'
                data2 = r2.json()
                # print(data["status"],data2["status"])
                if data["status"] == 0 and data2["status"]==0:
                    # 把有用的 ak 放回队列
                    self.aks_q.put(ak)
                    for i in range(len(data["result"])):
                        #print(i,coordids[i],w84c[i],data["result"][i]["x"],data["result"][i]["y"],data2["result"][i]["x"],data2["result"][i]["y"])
                        self.valid_q.put({
                            "coordid": coordids[i],
                            "w84c": w84c[i],
                            "bd09mc": "{},{}".format(data["result"][i]["x"],data["result"][i]["y"]),
                            "bd09ll": "{},{}".format(data2["result"][i]["x"],data2["result"][i]["y"])
                        })
                    break
                else:
                    print(data)
            except Exception as e:
                # print("\n Error: ", repr(e))
                traceback.print_exc()

    # 线程1： 访问 url 获取结果
    def producer(self):
        while self.groups_q.empty() is False:
            group = self.groups_q.get()
            self.req(group)
    
    # 线程2：将线程1的结果存入数据库
    def write(self):
        with open(r'/datadrive/lzy/Traffic/'+ self.city +'/'+ self.city +'Coordall.csv', 'w') as fileWriteObj:
            writer = csv.writer(fileWriteObj, lineterminator='\n')
            writer.writerow(['id','w84c','bd09mc','bd09ll','19_Tilex','19_Tiley','19_Pixelx','19_Pixely'])
            while (self.groups_q.empty() is False) or (self.valid_q.empty() is False):
                list_data = []
                if self.valid_q.empty() is False:
                    #print("write get\n")
                    data = self.valid_q.get()
                    list_data =  [data['coordid'], data['w84c'], data['bd09mc'], data['bd09ll']]
                    tileC = self.TileCoord(list_data[2],19)
                    list_data = list_data + tileC
                    # list_data =  [[_data['coordid'], _data['w84c'], _data['bd09mc'], _data['bd09ll']] for _data in data]
                    # list_data =  [[_data[0], _data[1], _data[2], _data[3]] for _data in data]
                    writer.writerow(list_data)
                    self.listall.append(list_data)
                #print(len(self.listall))
        print("run finish")
        print(len(self.listall))
#         name=['id','w84c','bd09mc','bd09ll']
#         test=pd.DataFrame(columns=name,data=self.listall)#数据有三列，列名分别为one,two,three
#         test.info()
#         test.to_csv('/datadrive/lzy/Traffic/TM/TMCoord1.csv',encoding='utf-8')
    
    # 线程3： 进度条显示
    def pbar(self):
        state = [0, 0]
        while (self.groups_q.empty() is False) or (self.valid_q.empty() is False) :
            if [self.lenth-self.groups_q.qsize(), self.valid_q.qsize()] != state:
                state = [self.lenth-self.groups_q.qsize(), self.valid_q.qsize()]
                print("Processing: {}/{}, time elapse: {:.0f}s, waiting for update: {}".format(self.lenth-self.groups_q.qsize(), self.lenth, time.time()-self.start, self.valid_q.qsize()))
                time.sleep(1)
                
    def run(self):
        ths =[]

        pbar_thread = Thread(target=self.pbar)
        pbar_thread.start()
        ths.append(pbar_thread)
        
        for _ in range(self.thread_num):
            producer_thread = Thread(target=self.producer)
            producer_thread.start()
            ths.append(producer_thread)

        write_thread = Thread(target=self.write)
        write_thread.start()
        ths.append(write_thread)

        # 阻塞主线程
        for th in ths:
            th.join()
        
        print("Time consume:", time.time()-self.start, "s")  

if __name__ == '__main__':
    text= Spider()
    text.run()
    print("Spider run finish")
    print(text.listall)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736629 entries, 0 to 736628
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   fid     736629 non-null  int64  
 1   r_id    736629 non-null  int64  
 2   r_type  736629 non-null  object 
 3   log     736629 non-null  float64
 4   lat     736629 non-null  float64
 5   pr_id   736629 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 33.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208724 entries, 0 to 208723
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   fid             208724 non-null  int64  
 1   roadid          208724 non-null  int64  
 2   roadtype        0 non-null       float64
 3   lat             208724 non-null  float64
 4   lng             208724 non-null  float64
 5   edm             208724 non-null  float64
 6   slope           197074 non-null  float6

FileNotFoundError: [Errno 2] No such file or directory: './BaiduAK.txt'

In [ ]:
import threading
from queue import Queue
from threading import Thread
import requests, urllib3
import sys, traceback
import time, os
import random
import numpy as np
import pandas as pd
import csv
import cv2
from skimage.morphology import disk
import skimage.filters.rank as sfr
import math
from concurrent.futures import ThreadPoolExecutor

class Spider():
    def __init__(self):
        self.city = 'BJ'
        self.thread_num = 1
        self.start = time.time()
        self.zoom = 19
        # 各种队列
        #self.trafficxy_q = self.get_trafficxys()# bd09mc坐标队列
        #self.TileCoord_q = self.coordTrans()# bd09mc与瓦片地图坐标转换后队列
        self.TileCoord_q = self.get_coordTrans()# bd09mc与瓦片地图坐标转换后队列
        self.all_lenth = self.TileCoord_q.qsize() # 队列总长度
        self.trafficimg_q = Queue() # traffic image queue to process
        self.finish_q = Queue() # 已经确认的队列
        #self.ippool_q = Queue() # 代理池队列
        for day in range(0,7):
            for hour in range(4,23):
                rootpath = "/datadrive/lzy/Traffic/"+self.city+"/"+self.city+"TrafficImg{0}_{1}".format(day,hour)
                rootpath_g = "/datadrive/lzy/Traffic/"+self.city+"/"+self.city+"TrafficImg{0}_{1}_graymin".format(day,hour)
                if not os.path.exists(rootpath):
                    os.mkdir(rootpath)
                if not os.path.exists(rootpath_g):
                    os.mkdir(rootpath_g)
        

    def get_trafficxys(self):
        df = pd.read_csv('/datadrive/lzy/Traffic/'+self.city+"/"+self.city+'Coord.csv')
        trafficxy_q = Queue()
        groups = np.array(df.iloc[:,[0,2]]).tolist()
        for group in groups:
            #print(group)
            trafficxy_q.put(group)
        return trafficxy_q

    
    def get_coordTrans(self):
        df_p = pd.read_csv('/datadrive/lzy/Traffic/'+self.city+"/"+self.city+'Coordall.csv')
        df_p.rename(columns={'id':'fid'},inplace=True)
        df_R = pd.read_csv('/datadrive/lzy/R/'+ self.city +'BSV_r.csv')
        df=pd.merge(df_p,df_R,on = 'fid',how='right')
        TileCoord_q = Queue()
        groups = np.array(df.iloc[:,[0,4,5,6,7]]).tolist()
        for group in groups:
            #print(group)
            TileCoord_q.put(group)
        return TileCoord_q
    
    
    def coordTrans(self):
        tilecoord_q = Queue()
        while self.trafficxy_q.empty() is False:
            group = self.trafficxy_q.get()
            pointx,pointy = group[1].split(",")
            Xtemp = float(pointx)* math.pow(2,zoom - 18)
            Ytemp = float(pointy)* math.pow(2,zoom - 18)
            tileX = math.floor(Xtemp / 256)
            tileY = math.floor(Ytemp / 256)
            pixelX = Xtemp - tileX * 256
            pixelY = Ytemp - tileY * 256
            list_data = [group[0],tileX, tileY, pixelX, pixelY]
            tilecoord_q.put(list_data)
        return tilecoord_q
             
    
    def req(self,day,hour,coord_data):
        # 访问 url 直到得到正确的访问结果
        #time.sleep(random.random())
        data = {
            "level": self.zoom,
            "x": coord_data[1],
            "y": coord_data[2],
            "day": day,
            "hour": hour,
            "t": time.time(),
            "v": "081",
            "smallflow": 1,
            "scaler": 1,
            }
        try:
            # request coordinations
            r = requests.get("https://its.map.baidu.com/traffic/HistoryService?", params=data,timeout=20)
            # url = "https://sp3.baidu.com/7_AZsjOpB1gCo2Kml5_Y_DAcsMJiwa/traffic/HistoryService?level=19&x=99538&y=27765&day=2&hour=11&t=1608801276170&v=081&smallflow=1&scaler=1"
            r.encoding='utf-8'
            rootpath = "/datadrive/lzy/Traffic/"+self.city+"/"+self.city+"TrafficImg{0}_{1}".format(day,hour)
            imgpath="/{0}_{1}.png".format(coord_data[1],coord_data[2])
            # print(day,hour,coord_data[0],coord_data[1],coord_data[2])
            open(rootpath + imgpath, 'wb').write(r.content)
            graypath = rootpath+"_graymin"+imgpath
            if not os.path.exists(graypath):
                img = cv2.imread(rootpath+imgpath, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img[img <1] = 255
                    imgmin = sfr.minimum(img,disk(10))
                    cv2.imwrite(graypath,imgmin)
            self.trafficimg_q.put("{},{}".format(rootpath,imgpath))
        except Exception as e:
            print("\n Error: ", repr(e), coord_data)
            traceback.print_exc()

                
    # 线程1： 访问 url 下载图片
    def producer(self):
        print("producer thread started.")
        while self.TileCoord_q.empty() is False:
            coord_data = self.TileCoord_q.get()
            threadPool = ThreadPoolExecutor(max_workers=4, thread_name_prefix="test_")
            for day in range(0,7):
                for hour in range(4,23):
                    rootpath = "/datadrive/lzy/Traffic/"+self.city+"/"+self.city+"TrafficImg{}_{}".format(day,hour)
                    imgpath = "/{0}_{1}.png".format(coord_data[1],coord_data[2])
                    if not os.path.exists(rootpath+imgpath):
                        threadPool.submit(self.req,day,hour,coord_data)
                    else:
                        self.trafficimg_q.put("{},{}".format(rootpath,imgpath))
            coord_data.append(imgpath)
            self.finish_q.put(coord_data)
        print("producer thread done")


    # 线程2： 更新数据库
    def imgprocess(self):
        print("img processor thread start\n")
        while (self.trafficimg_q.empty() is False) or (self.TileCoord_q.empty() is False):
            if self.trafficimg_q.empty() is False:
                path = self.trafficimg_q.get()
                rpth,imgpath = path.split(",")
                graypath = rpth+"_graymin"+imgpath
                if not os.path.exists(graypath):
                    img = cv2.imread(rpth+imgpath, cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        img[img <1] = 255
                        imgmin = sfr.minimum(img,disk(10))
                        cv2.imwrite(graypath,imgmin)
        print("img processor thread done")


    # 线程3： 爬取代理
    def get_traffics(self):
        print("traffic thread start\n")
        time.sleep(2)
        rpath = "/datadrive/lzy/Traffic/"+self.city+"/"+self.city+"Traffic"
        with open(rpath+'_0.csv', 'w') as fileWriteObj0, open(rpath+'mean.csv', 'w') as fileWriteObj_m, \
        open(rpath+'_1.csv', 'w') as fileWriteObj1, open(rpath+'_2.csv', 'w') as fileWriteObj2, \
        open(rpath+'_3.csv', 'w') as fileWriteObj3, open(rpath+'_4.csv', 'w') as fileWriteObj4, \
        open(rpath+'_5.csv', 'w') as fileWriteObj5, open(rpath+'_6.csv', 'w') as fileWriteObj6:
            writer0 = csv.writer(fileWriteObj0, lineterminator='\n')
            writer1 = csv.writer(fileWriteObj1, lineterminator='\n')
            writer2 = csv.writer(fileWriteObj2, lineterminator='\n')
            writer3 = csv.writer(fileWriteObj3, lineterminator='\n')
            writer4 = csv.writer(fileWriteObj4, lineterminator='\n')
            writer5 = csv.writer(fileWriteObj5, lineterminator='\n')
            writer6 = csv.writer(fileWriteObj6, lineterminator='\n')
            writer_m = csv.writer(fileWriteObj_m, lineterminator='\n')
            writer0.writerow(['id','0_4','0_5','0_6','0_7','0_8','0_9','0_10','0_11','0_12','0_13','0_14','0_15','0_16','0_17','0_18','0_19','0_20','0_21','0_22'])
            writer1.writerow(['id','1_4','1_5','1_6','1_7','1_8','1_9','1_10','1_11','1_12','1_13','1_14','1_15','1_16','1_17','1_18','1_19','1_20','1_21','1_22'])
            writer2.writerow(['id','2_4','2_5','2_6','2_7','2_8','2_9','2_10','2_11','2_12','2_13','2_14','2_15','2_16','2_17','2_18','2_19','2_20','2_21','2_22'])
            writer3.writerow(['id','3_4','3_5','3_6','3_7','3_8','3_9','3_10','3_11','3_12','3_13','3_14','3_15','3_16','3_17','3_18','3_19','3_20','3_21','3_22'])
            writer4.writerow(['id','4_4','4_5','4_6','4_7','4_8','4_9','4_10','4_11','4_12','4_13','4_14','4_15','4_16','4_17','4_18','4_19','4_20','4_21','4_22'])
            writer5.writerow(['id','5_4','5_5','5_6','5_7','5_8','5_9','5_10','5_11','5_12','5_13','5_14','5_15','5_16','5_17','5_18','5_19','5_20','5_21','5_22'])
            writer6.writerow(['id','6_4','6_5','6_6','6_7','6_8','6_9','6_10','6_11','6_12','6_13','6_14','6_15','6_16','6_17','6_18','6_19','6_20','6_21','6_22'])
            writer_m.writerow(['id','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22'])
            while (self.TileCoord_q.empty() is False) or (self.finish_q.empty() is False):
                if self.finish_q.empty() is False:
                    imginfo = self.finish_q.get()
                    #print(imginfo)
                    meantraffic = []
                    traffic0 = []
                    traffic1 = []
                    traffic2 = []
                    traffic3 = []
                    traffic4 = []
                    traffic5 = []
                    traffic6 = []
                    traffic0.append(imginfo[0])
                    traffic1.append(imginfo[0])
                    traffic2.append(imginfo[0])
                    traffic3.append(imginfo[0])
                    traffic4.append(imginfo[0])
                    traffic5.append(imginfo[0])
                    traffic6.append(imginfo[0])
                    meantraffic.append(imginfo[0])
                    for hour in range(4,23):
                        traffic = []
                        traffict = []
                        for day in range(0,7):
                            rootpath = "/datadrive/lzy/Traffic/"+self.city+"/"+self.city+"TrafficImg{0}_{1}".format(day,hour)
                            img = cv2.imread(rootpath+imginfo[5], cv2.IMREAD_GRAYSCALE)
                            if img is not None:
                                gray = img[255-int(imginfo[3]),int(imginfo[4])]
                                traffict.append(gray)
                                if gray == 105:
                                    traffic.append(0.9)
                                elif gray == 134:
                                    traffic.append(0.7)
                                elif gray == 215:
                                    traffic.append(0.4)
                                else:
                                    traffic.append(0.1)
                            else:
                                traffict.append(255)
                                traffic.append(0.1)
                        mean = np.mean(traffic)
                        traffic0.append(traffict[0])
                        traffic1.append(traffict[1])
                        traffic2.append(traffict[2])
                        traffic3.append(traffict[3])
                        traffic4.append(traffict[4])
                        traffic5.append(traffict[5])
                        traffic6.append(traffict[6])
                        meantraffic.append(mean)
                    writer_m.writerow(meantraffic)
                    writer0.writerow(traffic0)
                    writer1.writerow(traffic1)
                    writer2.writerow(traffic2)
                    writer3.writerow(traffic3)
                    writer4.writerow(traffic4)
                    writer5.writerow(traffic5)
                    writer6.writerow(traffic6)
        print("traffic thread done")

        
    # 线程4： 进度条显示
    def pbar(self):
        while (self.TileCoord_q.empty() is False) or (self.finish_q.empty() is False) or (self.trafficimg_q.empty() is False):
            print("Processing: {}/{}, time elapse: {:.0f}s, waiting for imgprocess:{}, waiting for get traffic from img: {}".format(self.all_lenth-self.TileCoord_q.qsize(), self.all_lenth, time.time()-self.start, self.trafficimg_q.qsize(), self.finish_q.qsize()))
            time.sleep(5)

    def run(self):
        ths =[]

        pbar_thread = Thread(target=self.pbar)
        pbar_thread.start()
        ths.append(pbar_thread)

        CoordTiles_thread = Thread(target=self.get_coordTrans)
        CoordTiles_thread.start()
        ths.append(CoordTiles_thread)

        for _ in range(self.thread_num):
            producer_thread = Thread(target=self.producer)
            producer_thread.start()
            ths.append(producer_thread)

#         threadPoolimg = ThreadPoolExecutor(max_workers=4, thread_name_prefix="img_")
#         process_thread = Thread(target=threadPoolimg.submit(self.imgprocess))
#         process_thread.start()
#         ths.append(process_thread)
        
        threadPoolimg = ThreadPoolExecutor(max_workers=3, thread_name_prefix="Traffic_")
        traffic_thread = Thread(target=threadPoolimg.submit(self.get_traffics))
        traffic_thread.start()
        ths.append(traffic_thread)

        # 阻塞主线程
        for th in ths:
            th.join()

        print("Time consume:", time.time()-self.start, "s")  

if __name__ == '__main__':
    Spider().run()

Processing: 368676/368676, time elapse: 253899s, waiting for imgprocess:49017131, waiting for get traffic from img: 0
Processing: 368676/368676, time elapse: 253904s, waiting for imgprocess:49017131, waiting for get traffic from img: 0


In [4]:
import pandas as pd

city = 'BJ'
df_info = pd.read_csv('/datadrive/lzy/Traffic/'+city+"/"+city+'PtDB.csv',names = ["id","Coordpairid","rid","rfclass","lng","lat","code","Coordpair","dem","slope","aspect"])
df_T = pd.read_csv('/datadrive/lzy/Traffic/'+city+"/"+city+'Trafficmean.csv')
df_T.info()
df_info.info()
df_T["id"] = df_T["id"].astype(int)
for i in range(4,23):
    df_T[str(i)] = df_T[str(i)].map(lambda x: 1-x)
df = pd.merge(df_info,df_T,on = "id",how = "left")
df.info()
trans={"4":1,"5":1,"6":0.9,"7":0.9,"8":0.9,"9":0.9,"10":0.9,"11":0.9,"12":0.9,"13":0.9,"14":0.9,"15":0.9,"16":0.9,"17":0.9,"18":0.9,"19":0.9,"20":0.9,"21":0.9,"22":0.9}
df = df.fillna(value=trans)
df = df.fillna(axis=0,method='ffill')
df_out = df.iloc[:,[0,2,3,4,5,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]]
# df.columns = ["pid","Coordpair","rfid","interval","rfid_2","rid","code","rfclass","name","ref","oneway","maxspeed","layer","bridge","tunnel","lng","lat","dem","slope","aspect"]
df_out.to_csv('/datadrive/lzy/Traffic/'+city+"/"+city+'AllTraffic.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368676 entries, 0 to 368675
Data columns (total 20 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      368676 non-null  float64
 1   4       368676 non-null  float64
 2   5       368676 non-null  float64
 3   6       368676 non-null  float64
 4   7       368676 non-null  float64
 5   8       368676 non-null  float64
 6   9       368676 non-null  float64
 7   10      368676 non-null  float64
 8   11      368676 non-null  float64
 9   12      368676 non-null  float64
 10  13      368676 non-null  float64
 11  14      368676 non-null  float64
 12  15      368676 non-null  float64
 13  16      368676 non-null  float64
 14  17      368676 non-null  float64
 15  18      368676 non-null  float64
 16  19      368676 non-null  float64
 17  20      368676 non-null  float64
 18  21      368676 non-null  float64
 19  22      368676 non-null  float64
dtypes: float64(20)
memory usage: 56.3 MB
<class 'pan

In [5]:
df_out

,id,rid,rfclass,lng,lat,dem,slope,aspect,4,5,...,13,14,15,16,17,18,19,20,21,22
0,1,4231222,primary,116.389467,39.905764,38.0,2.865651,118.064003,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
1,2,4231222,primary,116.389489,39.905265,33.0,7.793719,129.341599,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
2,3,4231222,primary,116.389511,39.904765,22.0,12.489513,122.537949,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
3,4,4231222,primary,116.389532,39.904266,17.0,10.220832,94.299652,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
4,5,4231222,primary,116.389554,39.903766,21.0,8.070035,56.693130,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736625,736625,641311622,unclassified,116.179974,40.219456,63.0,1.638770,172.411743,1.0,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
736626,736626,641311622,unclassified,116.180108,40.218974,61.0,0.547748,180.000000,1.0,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
736627,736627,641311622,unclassified,116.180241,40.218492,61.0,0.547748,180.000000,1.0,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
736628,736628,641311622,unclassified,116.182446,40.222260,67.0,0.362908,45.000000,1.0,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9


In [ ]:
# --coding:utf-8
# @author:Liu Ziyu
# Date:2020-04-22

import os
import glob
import numpy as np
import math
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import time
import csv

SConst = 1367
beta = 0.5
Div_skyazi = 16
day = [31,28,31,30,31,30,31,31,30,31,30,31]
# optical air mass from Smithsonian meteorological tables v.114
m_thita = [5.6,6.18,6.88,7.77,8.9,10.39,12.44,15.36,19.79,26.96]

# get all pano files under a rootpath
def get_allpano(cwd):
    result = []
    get_dir = os.listdir(cwd)  
    for i in get_dir:          
        sub_dir = os.path.join(cwd,i)  
        if os.path.isdir(sub_dir):     
            if sub_dir.endswith("_pano"):
                result.append(i)
    return result


# get all BSV files under a rootpath
def get_allBSV(cwd):
    result = []
    get_dir = os.listdir(cwd)  
    for i in get_dir:          
        sub_dir = os.path.join(cwd,i)  
        if os.path.isdir(sub_dir):     
            if sub_dir.endswith("BSV"):
                result.append(i)
    return result

# get all img under a rootpath
def get_allimg(cwd):
    paths = glob.glob(os.path.join(cwd, '*.png'))
    paths.sort()
    return paths


# extract the roadid, lat, lon from the imgname and return as array
def get_write_imginfo(imgname):
    imgname = imgname.rsplit('/',1)[1]
    temp = imgname.split('_')
    roadid = temp[0]
    lat = temp[1]
    lng = temp[2].rsplit('.',1)[0]
    return [roadid,lat,lng]


#split sunmaparr for center points in a year by month
def split_sunmaparr_c(sunmaparr_cpath):
    sunmaparr_c = np.loadtxt(sunmaparr_cpath,dtype=np.float,delimiter=',',encoding='utf-8')
    # split the sunmap by months
    intervalarrm = np.arange(2,13,1)
    split_atmc = sunmaparr_c[:, 0].searchsorted(intervalarrm)
    sunmaparr_c = np.split(sunmaparr_c, split_atmc)
    return sunmaparr_c

def split_skymaparr(skymap_path):
    skymaparr = np.loadtxt(skymap_path,dtype=np.float,delimiter=',',encoding='utf-8',skiprows=1)
    # split the sunmap by months
    intervalarrm = np.arange(1,128,1)
    split_at = skymaparr[:, 1].searchsorted(intervalarrm)
    skymaparr = np.split(skymaparr, split_at)
    return skymaparr

#split sunmaparr for each hour interval in a year by month and day hour   
def split_sunmaparr_y(sunmaparr_ypath):
    sunmaparr_y = np.loadtxt(sunmaparr_ypath,dtype=np.float,delimiter=',',encoding='utf-8',skiprows=1)
    intervalarrm = np.arange(2,13,1)
    split_atmy = sunmaparr_y[:, 0].searchsorted(intervalarrm)
    sunmaparr_y = np.array(np.split(sunmaparr_y, split_atmy))
    # split the sunmap by hour interval
    for i in range(0,len(sunmaparr_y)):
        # get the sunrise and sunset time for each day
        maxh = sunmaparr_y[i][:,3].max()
        minh = sunmaparr_y[i][:,3].min()
        # print(minh,maxh)
        if math.modf(maxh)[0] > 0.5:
            maxh = round(maxh)
        elif math.modf(maxh)[0] < 0.5:
            maxh = math.ceil(maxh)-0.5
        if math.modf(minh)[0] > 0.5:
            minh = round(minh)
        elif math.modf(minh)[0] < 0.5:
            minh = round(minh) + 0.5
        else:
            minh = minh + 0.5
        # sort the array by hour inter
        intervalarr = np.arange(minh, maxh, 0.5, dtype = float)
        temparr = np.array(sorted(sunmaparr_y[i],key=lambda x:(x[3])))
        split_ath = temparr[:, 3].searchsorted(intervalarr)
        sunmaparr_y[i] = np.split(temparr, split_ath)
#         print(sunmaparr_y[i])
    return sunmaparr_y


def get_gap(img,sunmapy_arr,hour_interval):
    blue_rgb = np.array([6, 230, 230],dtype = np.float)
    gap_sunmap = []
    hour_sundur = np.empty((len(sunmapy_arr)), dtype = np.float)
    for j in range(0,len(sunmapy_arr)):
#         print(len(sunmapy_arr))
        sundur = 0.0
        hour_gaparr = np.empty((len(sunmapy_arr[j])), dtype = np.float)
        for k in range(0,len(sunmapy_arr[j])):
#             print(len(sunmapy_arr[j]))
            count = 0
            ptnum = sunmapy_arr[j][k].shape[0]
            for pt in range(0,ptnum):
                x = int(sunmapy_arr[j][k][pt][9])
                y = int(sunmapy_arr[j][k][pt][10])
                if((image[x-1][y-1]*255 == blue_rgb).all()):
                    count += 1
            #print(count,ptnum)
            hour_gaparr[k] = count / ptnum * 1.0
            sundur += hour_gaparr[k] * hour_interval * len(sunmapy_arr[j])
        gap_sunmap.append(hour_gaparr)
        hour_sundur[j] = sundur
    return gap_sunmap, hour_sundur


def get_skygap(img,skymap_arr):
    blue_rgb = np.array([6, 230, 230],dtype = np.float)
    gap_skymap = []
#     hour_sundur = np.empty((len(sunmapy_arr)), dtype = np.float)
    for j in range(0,len(skymap_arr)):
        count = 0
        ptnum = skymap_arr[j].shape[0]
        for pt in range(0,ptnum): 
            x = int(skymap_arr[j][pt][4])
            y = int(skymap_arr[j][pt][5])
            if((image[x-1][y-1]*255 == blue_rgb).all()):
                count += 1
            #print(count,ptnum)
        gap_skymap.append(count / ptnum * 1.0)
    return gap_skymap


def deg2rad_G(data):
    if data == '':
        data = 0
    else:
        data = math.radians(float(data))
    return data


if __name__ == "__main__": 
    city = ['BJ','YA','TM','ZJ','SH','CD','BS','BJ','CQ','AL','FCG','JL','CD','JQ','NJZ','LS','QQHE','XA','ZT','HEB','HZ','SZ','TM','MDJ','SH','WH']
    cityi = 0
    # AL_30.3_81.1sunmapcenter.csv fcg_21.7_108.4sunmapcenter.csv JL_43.9_126.6sunmapcenter.csv CD_30.6_104.1sunmap.csv
    # JQ_39.7_98.5sunmap.csv NJZ_25.8_98.9sunmap.csv LS_29.7_91.1sunmap.csv QQHE_47.4_123.9sunmap XA_34.3_108.9sunmap.csv
    # ZJ_21.3_110.4sunmap ZT_27.3_103.7sunmap.csv HEB_45.8_126.5sunmap.csv HZ_24.4_111.6sunmap.csv SZ_22.5_114.1sunmap.csv
    # TM_30.7_113.2sunmap.csv MDJ_44.6_129.6sunmap.csv SH_31.2_121.1sunmap.csv CQ_29.5_106.5sunmap BJ_39.9_115.2sunmap
    # BS_42_126.4sunmap.csv YA_30_103.1sunmap
    rootpath = '/datadrive/lzy/Pano/'
    trafficpath = '/datadrive/lzy/Traffic/'+city[cityi]+'/'+city[cityi]+'AllTraffic.csv'
    cloudpath = '/datadrive/lzy/Traffic/'+city[cityi]+'/'+city[cityi]+'cloudy.csv'
    skymappath = '/datadrive/lzy/skymap_az.csv'
    outpath = '/datadrive/lzy/R_new/'
    data = np.loadtxt(trafficpath,dtype=np.str,delimiter=',',encoding='utf-8')
    cloudy =  np.loadtxt(cloudpath,dtype=np.str,delimiter=',',encoding='utf-8')
    # read the sunmap for centerpoint and all for given time interval like 0.05
    sunmapc_path = '/datadrive/lzy/Traffic/'+city[cityi]+'/BJ_39.9_115.2sunmapcenter.csv'
    sunmapy_path = '/datadrive/lzy/Traffic/'+city[cityi]+'/BJ_39.9_115.2sunmap.csv'
    sunmapc_arr = split_sunmaparr_c(sunmapc_path)
    sunmapy_arr = split_sunmaparr_y(sunmapy_path)
    skymap_arr = split_skymaparr(skymappath)
    sky_num = len(skymap_arr)
    
    # skymap: weight the proportion of diffuse radiation originating in a given sky sector relative to all sectors 
    skyweight_arr = []
    # the thita and azimuth of center point for each sector
    skythitac_arr = []
    skyazimuthc_arr = []
    for nsector in range(0,sky_num):
        thita_max = np.max(skymap_arr[nsector][:,15])
        thita_min = np.min(skymap_arr[nsector][:,15])
        thita_avg = np.mean(skymap_arr[nsector][:,15])
        azimuth_avg = np.mean(skymap_arr[nsector][:,16])
        # Weightθ,α = (cosθ2- cosθ1) / Divazi    (7)
        skyweight = (math.cos(math.radians(thita_min))-math.cos(math.radians(thita_max)))/Div_skyazi
        skyweight_arr.append(skyweight)
        skythitac_arr.append(math.radians(thita_avg))
        skyazimuthc_arr.append(math.radians(azimuth_avg))          
    
#     BSVfile = get_allBSV(rootpath)
    BSVfile = ['BJBSV']
    for rootf in BSVfile:
        panopath = rootpath + rootf
        outname = outpath + rootf + '_r.csv'
        outfile = open(outname, 'w' , newline="")
        writer = csv.writer(outfile, lineterminator='\n')
        writer.writerow(['fid','roadid','roadtype','lat','lng','edm','slope','aspect','dir_nt','dir_weighted','dif_nt','dif_weighted','total_nt','total_weighted'])
        panofile = get_allpano(panopath)
        for panof in panofile:
            imgpath = panopath + '/' + panof
            print(imgpath)
            imgname = get_allimg(imgpath)
#             imgname = get_allimg("/datadrive/urbanplayground/Streetview/SegDone/ALBSV/SegPic/ALBSV5_p_pano")
            # sparse image name
            for img in imgname:
                #print(data)
                imginfo = get_write_imginfo(img)
                f_id = int(imginfo[0])
                image = mpimg.imread(img)
                #plt.imshow(image)
                #plt.show()
                result = get_gap(image,sunmapy_arr,0.5)
                gap_sunmap = result[0]
#                 print(gap_sunmap)
                sundur_img = result[1]
                gap_sky = get_skygap(image,skymap_arr)
                # get the elevation of this image
                Elev = data[f_id,5]
                if Elev == '' or Elev == -1:
                    Elev = float(data[f_id-1,5])
                else:
                    Elev = float(Elev)
                # print(data[f_id,5])
                # Elev = float(data[f_id,5])
                # slope Gz aspect Ga
                Gz = deg2rad_G(data[f_id,6])
                Ga = deg2rad_G(data[f_id,7])
                # m(θ) is the relative optical path length
                mz_y = []
                AngIn_y = []
                # every sector Direct radiation workday
                Dir_ywork = []
                # each month of the total direct radiation workday
                Dir_monwork = []
                # the total direct of the year workday
                Dir_allwork = 0
                # for weekend
                Dir_ywkend = []
                Dir_monwkend = []
                Dir_allwkend = 0
                # without traffic
                Dir_yntraff = []
                Dir_monntraff = []
                Dir_allntraff = 0
                # weighted
                Dir_yweighted = []
                Dir_monweighted = []
                Dir_allweighted = 0
                # for diffuse radiation
                Rglb_work = 0
                Rglb_wkend = 0
                Rglb_ntraff = 0
                Rglb_weighted = 0
                for mon in range(0,len(sunmapc_arr)): 
                    Pdif = float(cloudy[mon][1])
                    mz_m = []
                    AngIn = []
                    # dir 
                    Dir_work = []
                    Dir_wkend = []
                    Dir_ntraff = []
                    Dir_weighted = []
                    # dir month tempt workday
                    Dir_monworkt = 0
                    Dir_monwkendt = 0
                    Dir_monntrafft = 0
                    Dir_monweightedt = 0
                    for num in range(0,len(sunmapc_arr[mon])):
                        thita = math.radians(90-sunmapc_arr[mon][num][2])
                        alfa = math.radians(sunmapc_arr[mon][num][3])
                        hournow = int(math.floor(sunmapc_arr[mon][num][1]))
#                         if hournow < 5 :
#                             hournow = 5
#                         elif hournow > 20:
#                             hournow = 20
                        # two calculate methods by thita copared to 80 degree
                        if thita < 80:
                            # For zenith angles less than 80 degree,
                            # m(θ) = EXP(-0. 000118 * Elev - 1. 638 * 10^-9 * Elev^2) /cos(θ)
                            mz_t = math.exp(-0.000118 * Elev - 1.638 * math.pow(10,-9) * math.pow(Elev,2))/math.cos(thita)
                        else:
                            # get from 
                            index = int(round(thita))-80
                            if index == 10:
                                mz_t = m_thita[9]
                            else:
                                mz_t = m_thita[index]
                        # Angle of incidence (AngInSky θ,α) between the intercepting surface and a given sky sector with a centroid at zenith angle θ and azimuth angle α
                        # AngIn θ,α = acos[Cos(θ)*Cos(Gz)+Sin(θ)*Sin(Gz)*Cos(α-Ga)]
                        angin_t = math.acos(math.cos(thita)*math.cos(Gz)+math.sin(thita)*math.sin(Gz)*abs(math.cos(alfa-Ga)))
                        # calculate Direct rdiation
                        # Dirθ,α = SConst * β^m(θ) * SunDurθ,α * SunGapθ,α * cos(AngInθ,α)
                        gap_count = len(gap_sunmap[mon])
                        if num == gap_count or num > gap_count:
                            gap_tem = gap_sunmap[mon][gap_count-1]
                        else:
                            gap_tem = gap_sunmap[mon][num]
                        Dir_ntrafft = SConst * math.pow(beta,mz_t) * day[mon]*0.5 * gap_tem * math.cos(angin_t)*(1-Pdif)
                        Dir_weightedt = Dir_ntrafft * float(data[f_id,4+hournow])
#                         Dir_workt = Dir_ntrafft * float(data[f_id,3+hournow])
#                         Dir_wkendt = Dir_ntrafft * float(data[f_id,19+hournow])
#                         Dir_weightedt = Dir_workt * 5/7 + Dir_wkendt * 2/7
#                         Dir_monworkt += Dir_workt
#                         Dir_monwkendt += Dir_wkendt
                        Dir_monntrafft += Dir_ntrafft
                        Dir_monweightedt += Dir_weightedt
                        mz_m.append(mz_t)
#                         mz_sum += mz_t * Pdif * day[mon] * 0.5
                        # prepare for diffuse radiation for Rglb
                        # Rglb = (SConst Σ(βm(θ))) / (1 - Pdif)
                        Rglb_ntrafft = SConst * math.pow(beta,mz_t) * Pdif * 0.5 / (1-Pdif)
                        Rglb_weightedt = Rglb_ntrafft * float(data[f_id,4+hournow])
#                         Rglb_workt = Rglb_ntrafft * float(data[f_id,4+hournow])
#                         Rglb_wkendt = Rglb_ntrafft * float(data[f_id,19+hournow])
                        Rglb_ntraff += Rglb_ntrafft
#                         Rglb_work += Rglb_workt
#                         Rglb_wkend += Rglb_wkendt
                        Rglb_weighted += Rglb_weightedt
                        AngIn.append(angin_t)
                        Dir_ntraff.append(Dir_ntrafft)
#                         Dir_work.append(Dir_workt)
#                         Dir_wkend.append(Dir_wkendt)
                        Dir_weighted.append(Dir_weightedt)
                    Dir_yntraff.append(Dir_ntraff)
#                     Dir_ywork.append(Dir_work)
#                     Dir_ywkend.append(Dir_wkend)
                    Dir_yweighted.append(Dir_weighted)
#                     Dir_monwork.append(Dir_monworkt)
                    Dir_allntraff += Dir_monntrafft
#                     Dir_allwork += Dir_monworkt
#                     Dir_allwkend += Dir_monwkendt
                    Dir_allweighted += Dir_monweightedt
                    mz_y.append(mz_m)
                    AngIn_y.append(AngIn)
#                 Rglb_weighted = Rglb_work * 5/7 + Rglb_wkend *2/7
#                 print('Dir:',Dir_allntraff,Dir_allwork,Dir_allwkend,Dir_allweighted)
#                 print("Rglb:",Rglb_ntraff,Rglb_work,Rglb_wkend,Rglb_weighted)
                skyAngIn = []
                #  Difθ,α = Rglb * Pdif * Dur * SkyGapθ,α * Weightθ,α * cos(AngInθ,α) 
                Dif_secntraff = []
#                 Dif_secwork = []
#                 Dif_secwkend = []
                Dif_secweighted = []
                Dif_sumntraff = 0
#                 Dif_sumwork = 0
#                 Dif_sumwkend = 0
                Dif_sumweighted = 0
                for nsec in range(0,sky_num):
                    sky_thita = skythitac_arr[nsec]
                    skyangin_t = math.acos(math.cos(sky_thita)*math.cos(Gz)+math.sin(sky_thita)*math.sin(Gz)*abs(math.cos(skyazimuthc_arr[nsec]-Ga)))
                    tempt = gap_sky[nsec] * skyweight_arr[nsec] * math.cos(skyangin_t)
#                     print(Rglb_ntraff,gap_sky[nsec],skyweight_arr[nsec],math.cos(skyangin_t))
#                     time.sleep(1)
                    Dif_ntrafft = Rglb_ntraff * tempt
#                     Dif_workt = Rglb_work * tempt
#                     Dif_wkendt = Rglb_wkend * tempt
                    Dif_weightedt = Rglb_weighted * tempt
                    skyAngIn.append(skyangin_t)
                    Dif_secntraff.append(Dif_ntrafft)
#                     Dif_secwork.append(Dif_workt)
#                     Dif_secwkend.append(Dif_wkendt)
                    Dif_secweighted.append(Dif_weightedt)
                    Dif_sumntraff += Dif_ntrafft
#                     Dif_sumwork += Dif_workt
#                     Dif_sumwkend += Dif_wkendt
                    Dif_sumweighted += Dif_weightedt
#                 print('Dif:',Dif_sumntraff,Dif_sumwork,Dif_sumwkend,Dif_sumweighted)
                writer.writerow([f_id,data[f_id,1],data[f_id,2],imginfo[1],imginfo[2],Elev,data[f_id,6],data[f_id,7],
                                 Dir_allntraff,Dir_allweighted,Dif_sumntraff,Dif_sumweighted,
                                 Dir_allntraff+Dif_sumntraff,Dir_allweighted+Dif_sumweighted])
        outfile.close()
        print('finish')

/home/ziyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


/datadrive/lzy/Pano/BJBSV/BJBSV2_p_pano
/datadrive/lzy/Pano/BJBSV/BJBSV3_p_pano
/datadrive/lzy/Pano/BJBSV/BJBSV4_p_pano
/datadrive/lzy/Pano/BJBSV/BJBSV6_p_pano
